In [2]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

In [3]:
spark = SparkSession.builder.appName("company").getOrCreate()

In [6]:
df = spark.read.csv(r'C:\Users\ngupt\OneDrive\Desktop\perschoolas\Spark\spark_demo\spark_files\CompanyABC_stock.csv',\
    header=True,inferSchema=True)

In [7]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [8]:
#1.2: Print out the first five rows.
df.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [11]:
#1.3: Create a new Dataframe column called “HV Ratio,” which will stimulate the ratio of the High price versus the total Volume of stock that was traded for a day. 
df_HV= df.withColumn("HV_Ratio",df['High']/df['Volume'])
df_HV.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV_Ratio|
+-------------------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215

In [14]:
#1.4: Find out on what day the stock price was the highest. (Hint: use the High column.)
#df.select('date').filter('High'==max('High'))
df.orderBy(df["High"].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

In [15]:
#1.5: What is the average (mean) closing price? (Hint: Use the Close column.)
df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [16]:
#1.6: What is the maximum and minimum volume of stock traded? (Hint: Use the Volume column).
df.select(max("Volume"),min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [19]:
#1.7: For how many days was the closing value less than 70 dollars? (Hint: Use the count() method.)
df.filter(df['Close'] < 70).count()

397

In [23]:
#1.8: What percentage of the time was the High greater than 80 dollars?
 #(Hint: (Number of Days High>80)/(Total Days in the dataset)]

count_high=count=df.filter(df["High"]>80).count()
per_high=(count_high/df.count())*100
per_high

9.141494435612083

#1.9: Create a database named CompanyABC_DB using MariaDB(HeidiSQL).  
1.10: Load/Write CompanyABC stock.csv file data into CompanyABC_DB database  from SparkSQL Dataframe. You can specify any table name for that file.

In [24]:
df.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/companyabc_db") \
  .option("dbtable", "companyABC_forSparkDemo") \
  .option("user", "root") \
  .option("password", "password") \
  .save()


2.1: Load/Read both CompanyABC sales datasets (Sales_April_2019.csv and Sales_February_2019.csv) into SparkSQL Dataframe ( hint read() ).
You can inspect data by using the lines below: 


In [34]:
df_sales=spark.read.csv(['spark_files/Sales_February_2019.csv','spark_files/Sales_April_2019.csv'],inferSchema=True,header='True')

In [36]:
df_sales.columns

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address']

In [37]:
df_sales.printSchema()
df_sales.describe().show()
df_sales.show()

root
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)

+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|summary|          Order ID|     Product|   Quantity Ordered|        Price Each|    Order Date|    Purchase Address|
+-------+------------------+------------+-------------------+------------------+--------------+--------------------+
|  count|             30275|       30328|              30275|             30275|         30328|               30328|
|   mean|173816.63355904212|        null| 1.1239966969446737| 183.7621383980421|          null|                null|
| stddev|14902.179833784816|        null|0.43431295924749175|328.79347229907665|          null|                null|
|    min|            150502|20i

Now, create a new Dataframe column called “Total price” and find the “Total price” of the Order for both sales 
Hint: (Total price = Price Each x Quantity Ordered)

In [40]:

df_sales=df_sales.withColumn("Total price",df_sales['Price Each']*df_sales['Quantity Ordered'])

In [42]:
df_sales.columns

['Order ID',
 'Product',
 'Quantity Ordered',
 'Price Each',
 'Order Date',
 'Purchase Address',
 'Total price']

In [47]:
df_sales.select(['Total Price']).show()

+-----------+
|Total Price|
+-----------+
|       23.9|
|      99.99|
|      600.0|
|      11.99|
|      11.99|
|      11.95|
|      99.99|
|      11.95|
|     1700.0|
|      11.99|
|      600.0|
|      14.95|
|     389.99|
|       3.84|
|      14.95|
|      150.0|
|      11.95|
|      600.0|
|      11.95|
|       2.99|
+-----------+
only showing top 20 rows



In [43]:
df_sales.show(5)

+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|Order ID|             Product|Quantity Ordered|Price Each|     Order Date|    Purchase Address|Total price|
+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
|  176558|USB-C Charging Cable|               2|     11.95| 4/19/2019 8:46|917 1st St, Dalla...|       23.9|
|  176559|Bose SoundSport H...|               1|     99.99| 4/7/2019 22:30|682 Chestnut St, ...|      99.99|
|  176560|        Google Phone|               1|     600.0|4/12/2019 14:38|669 Spruce St, Lo...|      600.0|
|  176560|    Wired Headphones|               1|     11.99|4/12/2019 14:38|669 Spruce St, Lo...|      11.99|
|  176561|    Wired Headphones|               1|     11.99| 4/30/2019 9:27|333 8th St, Los A...|      11.99|
+--------+--------------------+----------------+----------+---------------+--------------------+-----------+
only showing top 5 

 Load/Write sales data from SparkSQL Dataframe into CompanyABC_DB database. You can specify any n:ame to the table. Remember “Total price” must be recorded. 

In [44]:
df_sales.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/companyabc_db") \
  .option("dbtable", "companySales_forSparkDemo") \
  .option("user", "root") \
  .option("password", "password") \
  .save()